This notebook follows the one analysing the game data : https://github.com/Canisback/lolGameAnalysis/blob/master/simpleGameAnalysis.ipynb


In this notebook, we will focus on the data you will find in the timeline endpoint.

Import pantheon and asyncio, initiate Pantheon

In [1]:
from pantheon import pantheon
import asyncio

server = "euw1"
api_key = "RGAPI-XXXX"

panth = pantheon.Pantheon(server, api_key, True)

Create function to gather the data. Here there is a need to call a second endpoint for the timeline data. For convenience, I put the timeline data inside the game data.

In [2]:

async def getSummonerId(name):
    try:
        data = await panth.getSummonerByName(name)
        return (data['id'],data['accountId'])
    except Exception as e:
        print(e)


async def getRecentMatchlist(accountId):
    try:
        data = await panth.getMatchlist(accountId, params={"endIndex":10})
        return data
    except Exception as e:
        print(e)

async def getMatchAndTimeline(gameId):
    match, timeline = await asyncio.gather(*[
        panth.getMatch(gameId),
        panth.getTimeline(gameId)
    ])
    match['timeline'] = timeline
    return match
        
async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [getMatchAndTimeline(match['gameId']) for match in matchlist['matches']]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)

Get your games and timeline data

In [3]:
name = "Canisback"

loop = asyncio.get_event_loop()  

(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))
gamesDataRaw = loop.run_until_complete(getRecentMatches(accountId))
print(gamesDataRaw[0]['timeline'])

{'frames': [{'timestamp': 152, 'events': [], 'participantFrames': {'7': {'dominionScore': 0, 'level': 1, 'minionsKilled': 0, 'participantId': 7, 'currentGold': 500, 'xp': 0, 'totalGold': 500, 'teamScore': 0, 'position': {'x': 14486, 'y': 14291}, 'jungleMinionsKilled': 0}, '8': {'dominionScore': 0, 'level': 1, 'minionsKilled': 0, 'participantId': 8, 'currentGold': 500, 'xp': 0, 'totalGold': 500, 'teamScore': 0, 'position': {'x': 14277, 'y': 14223}, 'jungleMinionsKilled': 0}, '5': {'dominionScore': 0, 'level': 1, 'minionsKilled': 0, 'participantId': 5, 'currentGold': 500, 'xp': 0, 'totalGold': 500, 'teamScore': 0, 'position': {'x': 311, 'y': 649}, 'jungleMinionsKilled': 0}, '4': {'dominionScore': 0, 'level': 1, 'minionsKilled': 0, 'participantId': 4, 'currentGold': 500, 'xp': 0, 'totalGold': 500, 'teamScore': 0, 'position': {'x': 222, 'y': 471}, 'jungleMinionsKilled': 0}, '9': {'dominionScore': 0, 'level': 1, 'minionsKilled': 0, 'participantId': 9, 'currentGold': 500, 'xp': 0, 'totalGold

Like previsouly, the first thing we need is the participantId

In [4]:
def getParticipantIdFromAccountId(game,accountId):
    for participant in game['participantIdentities']:
        if participant['player']['currentAccountId'] == accountId:
            return participant['participantId']
        
participantId = getParticipantIdFromAccountId(gamesDataRaw[0],accountId)
participantId

5

The timeline data mostly consist in a snapshot of the game every minute. You have two types of data, partcipant data and events data.

Participant data gives some data every minute, like XP, Gold, position on the map : 

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gamesDataRaw[0]['timeline']['frames'][10]['participantFrames']['1'])

{   'currentGold': 2284,
    'dominionScore': 0,
    'jungleMinionsKilled': 60,
    'level': 7,
    'minionsKilled': 13,
    'participantId': 1,
    'position': {'x': 3754, 'y': 7867},
    'teamScore': 0,
    'totalGold': 4534,
    'xp': 3990}


Events data gives data about particuliar events in the game, like kills, building destruction, item purchase or skill levelup

In [6]:
pp.pprint(gamesDataRaw[0]['timeline']['frames'][1]['events'][0])
pp.pprint(gamesDataRaw[0]['timeline']['frames'][1]['events'][1])

{   'itemId': 1041,
    'participantId': 10,
    'timestamp': 2924,
    'type': 'ITEM_PURCHASED'}
{   'levelUpType': 'NORMAL',
    'participantId': 1,
    'skillSlot': 1,
    'timestamp': 2924,
    'type': 'SKILL_LEVEL_UP'}


To get the skill order of a player, you'll have to parse every event of every frame, and select the ones matching the 'SKILL_LEVEL_UP' type and the right participantId

In [7]:
def getSkillOrder(game,participantId):
    skillOrder = []
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == 'SKILL_LEVEL_UP' and event['participantId'] == participantId:
                skillOrder.append(str(event['skillSlot']))
    return " > ".join(skillOrder)

skillOrder = getSkillOrder(gamesDataRaw[0],participantId)
skillOrder

'1 > 3 > 2 > 3 > 3 > 4 > 3 > 1 > 3 > 1 > 4'

You can also replace number by skill letter

In [8]:
skillOrder.replace("1","Q").replace("2","W").replace("3","E").replace("4","R")

'Q > E > W > E > E > R > E > Q > E > Q > R'

The build order part is more complex as it has more data, and more useful data. First thing first, let's get the whole list of items purchased by a player

In [9]:
def getAllItems(game, participantId):
    itemsList = []
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == 'ITEM_PURCHASED' and event['participantId'] == participantId:
                itemsList.append(event['itemId'])
    return itemsList

getAllItems(gamesDataRaw[0],participantId)

[3302,
 2003,
 2003,
 2003,
 3340,
 1001,
 3097,
 2003,
 2003,
 2003,
 1028,
 3057,
 3401,
 3364,
 1029,
 3025,
 3009,
 2065,
 1033]

Another useful information can be the time in the game when the item was bought. We'll have a list of tuple with time and itemId.

In [10]:
def getAllItemsWithTime(game, participantId):
    itemsList = []
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == 'ITEM_PURCHASED' and event['participantId'] == participantId:
                itemsList.append((event['timestamp'],event['itemId']))
    return itemsList

getAllItemsWithTime(gamesDataRaw[0],participantId)

[(6257, 3302),
 (6620, 2003),
 (6785, 2003),
 (6983, 2003),
 (7511, 3340),
 (350333, 1001),
 (350696, 3097),
 (351125, 2003),
 (351290, 2003),
 (351455, 2003),
 (459171, 1028),
 (539529, 3057),
 (668894, 3401),
 (670808, 3364),
 (675495, 1029),
 (886671, 3025),
 (888816, 3009),
 (1127380, 2065),
 (1130648, 1033)]

Now we want it to be pretty, change the item to their name, and time to the format "minute:second". Like in the previous notebook, we will need the data for items.

In [11]:
import requests,json
response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.9.1/data/en_US/item.json')
itemRawData = json.loads(response.text)

itemIdToName = {}
for key,item in itemRawData['data'].items():
    itemIdToName[int(key)] = item['name']

def getPrettyTimeFromTimestamp(timestamp):
    seconds = int(timestamp/1000 %60)
    minutes = int(timestamp/1000 /60)
    return str(minutes)+":"+str(seconds)
    
def getAllItemsWithTimeAndPretty(game, participantId):
    itemsList = []
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == 'ITEM_PURCHASED' and event['participantId'] == participantId:
                itemsList.append((getPrettyTimeFromTimestamp(event['timestamp']),itemIdToName[event['itemId']]))
    return itemsList

getAllItemsWithTimeAndPretty(gamesDataRaw[0],participantId)

[('0:6', 'Relic Shield'),
 ('0:6', 'Health Potion'),
 ('0:6', 'Health Potion'),
 ('0:6', 'Health Potion'),
 ('0:7', 'Warding Totem (Trinket)'),
 ('5:50', 'Boots of Speed'),
 ('5:50', "Targon's Brace"),
 ('5:51', 'Health Potion'),
 ('5:51', 'Health Potion'),
 ('5:51', 'Health Potion'),
 ('7:39', 'Ruby Crystal'),
 ('8:59', 'Sheen'),
 ('11:8', 'Remnant of the Aspect'),
 ('11:10', 'Oracle Lens'),
 ('11:15', 'Cloth Armor'),
 ('14:46', 'Iceborn Gauntlet'),
 ('14:48', 'Boots of Swiftness'),
 ('18:47', "Shurelya's Reverie"),
 ('18:50', 'Null-Magic Mantle')]

Our last step in build order analysis is filtering the items. For example, we would like to know only the major items, those that are not consumable and can't be upgraded.

In [12]:
itemListFiltre = []
for key,item in itemRawData['data'].items():
    if not ("Consumable" in item['tags'] or "into" in item):
        itemListFiltre.append(int(key))
    #Don't forget items that Ornn can upgrade but can still be counted as major item 
    if "into" in item:
        if len(item['into']) == 1:
            if "requiredAlly" in itemRawData['data'][key]:
                 itemListFiltre.append(int(key))
        
def getMajorItemsWithTimeAndPretty(game, participantId):
    itemsList = []
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == 'ITEM_PURCHASED' and event['participantId'] == participantId:
                if event['itemId'] in itemListFiltre:
                    itemsList.append((getPrettyTimeFromTimestamp(event['timestamp']),itemIdToName[event['itemId']]))
    return itemsList

getMajorItemsWithTimeAndPretty(gamesDataRaw[0],participantId)

[('0:7', 'Warding Totem (Trinket)'),
 ('11:8', 'Remnant of the Aspect'),
 ('11:10', 'Oracle Lens'),
 ('14:46', 'Iceborn Gauntlet'),
 ('14:48', 'Boots of Swiftness'),
 ('18:47', "Shurelya's Reverie")]

Here you have two important information we can extract from the timeline data. Of course there are many more things to get from timeline data, but it's up to you.